# (R) 비트코인의 daily/hourly data를 바탕으로

- 거품 측정 (AR(1)의 alpha > 1인지 확인) 및 비트코인이 일반 금융자산이 거품상태일 때와 유사한 양상을 보이는지 확인. 
- GJR-GARCH(1,1)의 gamma 확인 
- 거래량 데이터를 이용해 ARMAX/GARCHX 모델 수립
- daily/hourly data에 따라 차이가 있는지 확인
